## Optimize CNN model using Keras Tuner

In [1]:
!pip install keras-tuner==1.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 2.4 MB/s eta 0:00:00


In [2]:
!pip install tensorflow==2.2.0

ERROR: Could not find a version that satisfies the requirement tensorflow==2.2.0 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0)
ERROR: No matching distribution found for tensorflow==2.2.0


In [21]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation

In [4]:
print(tf.__version__)

2.13.0


In [5]:
fashion_mnist=keras.datasets.fashion_mnist

In [6]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [7]:
train_images=train_images/255.0
test_images=test_images/255.0

In [8]:
train_images[0].shape

(28, 28)

In [9]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [12]:
print(train_images.shape)
print(test_images.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [22]:
print(train_labels.shape)
print(test_labels.shape)

(60000,)
(10000,)


In [17]:
#Method#1
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16), #Use the hp argument to define the hyperparameters during model creation.
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),

    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

In [18]:
from kerastuner import RandomSearch

In [19]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output_mnist',project_name="mnist_Fashion")

In [20]:
tuner_search.search(train_images,train_labels,epochs=10,validation_split=0.1)

Epoch 1/10
1688/1688 [==============================] - 20s 5ms/step - loss: 0.4241 - accuracy: 0.8498 - val_loss: 0.3111 - val_accuracy: 0.8885
Epoch 2/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2793 - accuracy: 0.8984 - val_loss: 0.2705 - val_accuracy: 0.9005
Epoch 3/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2312 - accuracy: 0.9151 - val_loss: 0.2574 - val_accuracy: 0.9073
Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1981 - accuracy: 0.9264 - val_loss: 0.2370 - val_accuracy: 0.9193
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1712 - accuracy: 0.9366 - val_loss: 0.2400 - val_accuracy: 0.9142
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1487 - accuracy: 0.9452 - val_loss: 0.2435 - val_accuracy: 0.9192
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1244 - accuracy: 0.9537 - val_loss: 0.2490 - val_accuracy

Epoch 1/10
1688/1688 [==============================] - 10s 5ms/step - loss: 0.4782 - accuracy: 0.8265 - val_loss: 0.3992 - val_accuracy: 0.8540
Epoch 2/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3849 - accuracy: 0.8579 - val_loss: 0.3896 - val_accuracy: 0.8542
Epoch 3/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3638 - accuracy: 0.8666 - val_loss: 0.3971 - val_accuracy: 0.8645
Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3505 - accuracy: 0.8727 - val_loss: 0.3889 - val_accuracy: 0.8683
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3352 - accuracy: 0.8779 - val_loss: 0.3694 - val_accuracy: 0.8673
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3287 - accuracy: 0.8780 - val_loss: 0.4080 - val_accuracy: 0.8590
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3183 - accuracy: 0.8835 - val_loss: 0.4547 - val_accuracy

Epoch 1/10
1688/1688 [==============================] - 11s 5ms/step - loss: 0.4111 - accuracy: 0.8499 - val_loss: 0.3103 - val_accuracy: 0.8835
Epoch 2/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2689 - accuracy: 0.9007 - val_loss: 0.2706 - val_accuracy: 0.8978
Epoch 3/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2228 - accuracy: 0.9161 - val_loss: 0.2406 - val_accuracy: 0.9102
Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1860 - accuracy: 0.9311 - val_loss: 0.2827 - val_accuracy: 0.9035
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1533 - accuracy: 0.9432 - val_loss: 0.2340 - val_accuracy: 0.9210
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1308 - accuracy: 0.9507 - val_loss: 0.2486 - val_accuracy: 0.9180
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1080 - accuracy: 0.9591 - val_loss: 0.2774 - val_accuracy

Epoch 1/10
1688/1688 [==============================] - 12s 6ms/step - loss: 0.4513 - accuracy: 0.8363 - val_loss: 0.3756 - val_accuracy: 0.8583
Epoch 2/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.3423 - accuracy: 0.8729 - val_loss: 0.3457 - val_accuracy: 0.8752
Epoch 3/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3193 - accuracy: 0.8822 - val_loss: 0.3348 - val_accuracy: 0.8772
Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3114 - accuracy: 0.8844 - val_loss: 0.4007 - val_accuracy: 0.8522
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2941 - accuracy: 0.8910 - val_loss: 0.3449 - val_accuracy: 0.8762
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2842 - accuracy: 0.8951 - val_loss: 0.3459 - val_accuracy: 0.8808
Epoch 7/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.2798 - accuracy: 0.8964 - val_loss: 0.3428 - val_accura

Epoch 1/10
1688/1688 [==============================] - 10s 5ms/step - loss: 0.4418 - accuracy: 0.8411 - val_loss: 0.3518 - val_accuracy: 0.8757
Epoch 2/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2912 - accuracy: 0.8930 - val_loss: 0.2734 - val_accuracy: 0.9005
Epoch 3/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2428 - accuracy: 0.9103 - val_loss: 0.2650 - val_accuracy: 0.9007
Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2090 - accuracy: 0.9220 - val_loss: 0.2779 - val_accuracy: 0.8965
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1834 - accuracy: 0.9324 - val_loss: 0.2459 - val_accuracy: 0.9108
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1576 - accuracy: 0.9407 - val_loss: 0.2656 - val_accuracy: 0.9097
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1356 - accuracy: 0.9492 - val_loss: 0.2563 - val_accuracy

In [23]:
tuner_search.results_summary()

In [24]:
model=tuner_search.get_best_models(num_models=1)[0]

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 80)        800       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 80)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 48)        34608     
                                                                 
 flatten (Flatten)           (None, 5808)              0         
                                                                 
 dense (Dense)               (None, 48)                278832    
                                                                 
 dense_1 (Dense)             (None, 10)                490       
                                                        

In [26]:
model.fit(train_images, train_labels, epochs=20, validation_split=0.1, initial_epoch=10)

Epoch 11/20
1688/1688 [==============================] - 10s 5ms/step - loss: 0.1084 - accuracy: 0.9596 - val_loss: 0.2775 - val_accuracy: 0.9215
Epoch 12/20
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0907 - accuracy: 0.9664 - val_loss: 0.3260 - val_accuracy: 0.9138
Epoch 13/20
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0804 - accuracy: 0.9696 - val_loss: 0.3005 - val_accuracy: 0.9167
Epoch 14/20
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0690 - accuracy: 0.9739 - val_loss: 0.3321 - val_accuracy: 0.9178
Epoch 15/20
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0587 - accuracy: 0.9780 - val_loss: 0.3724 - val_accuracy: 0.9160
Epoch 16/20
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0522 - accuracy: 0.9810 - val_loss: 0.3684 - val_accuracy: 0.9155
Epoch 17/20
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0442 - accuracy: 0.9835 - val_loss: 0.3962 - val_a

i am training above with initial_epochs=10 because already my model is trained till epoch=10

In [27]:
# Evaluate the best model.
loss, accuracy = model.evaluate(test_images, test_labels)
print('loss:', loss)
print('accuracy:', accuracy)

313/313 [==============================] - 1s 3ms/step - loss: 0.5052 - accuracy: 0.9095
loss: 0.5051743388175964
accuracy: 0.909500002861023


In [28]:
# save model
model.save('final_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
# make a prediction for a new image.
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model

# load and prepare the image
def load_image(filename):
 # load the image
 img = load_img(filename, grayscale=True, target_size=(28, 28))
 # convert to array
 img = img_to_array(img)
 # reshape into a single sample with 1 channel
 img = img.reshape(1, 28, 28, 1)
 # prepare pixel data
 img = img.astype('float32')
 img = img / 255.0
 return img

# load an image and predict the class
def run_example():
 # load the image
 img = load_image('sample_image.png')
 # load model
 model = load_model('final_model.h5')
 # predict the class
 result = model.predict(img)
 print(result[0])
 classes_x=np.argmax(result,axis=1)
 print(classes_x)

# entry point, run the example
run_example()

/usr/local/lib/python3.10/dist-packages/keras/src/utils/image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


1/1 [==============================] - 0s 63ms/step
[1.5819164e-04 2.4772216e-21 9.9984157e-01 2.1837572e-13 1.0911214e-07
 2.1612524e-17 1.1230359e-07 4.3432409e-25 1.1546351e-17 2.0208424e-18]
[2]
